# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [595]:
problem = '''You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem.   '''

## 2. Ask for parameters

In [596]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [597]:
#Show the resopnse in a formatted way
Markdown(response.text)

### Sets:
*  $I$: Set of items
*  $J$: Set of boxes

### Parameters:
*  $w_i$: Weight of item $i \in I$
*  $C$: Capacity of each box 
*  $N$: Upper bound on the number of boxes 

### Decision Variables:
*  $x_{ij}$: Binary variable equal to 1 if item  $i \in I$ is assigned to box $j \in J$, and 0 otherwise 
*  $y_j$: Binary variable equal to 1 if box $j \in J$ is used, and 0 otherwise 


# 2. Ask for objective

In [598]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [599]:
Markdown(response2.text)

Minimize $\sum_{j \in J} y_j$ 


# 3. Ask for constraints

In [600]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [601]:
Markdown(response3.text)

## Constraints:

1. **Each item must be assigned to exactly one box:**
   $$ \sum_{j \in J} x_{ij} = 1, \quad \forall i \in I$$

2. **Capacity constraint of each box:**
   $$ \sum_{i \in I} w_i x_{ij} \le C y_j, \quad \forall j \in J$$

3. **Limit on the number of boxes:**
   $$ \sum_{j \in J} y_j \le N$$

4. **Decision variables are binary:**
   $$ x_{ij} \in \{0,1\}, \quad \forall i \in I, \forall j \in J$$
   $$ y_j \in \{0,1\}, \quad \forall j \in J $$ 


## 3. Generate the pyomo code

In [602]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [603]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data (You can replace this with your actual data)
items = ['A', 'B', 'C', 'D']  # Set of items I
boxes = ['Box1', 'Box2', 'Box3']  # Set of boxes J
weights = {'A': 5, 'B': 3, 'C': 7, 'D': 2}  # Weights of items w_i
capacity = 10  # Capacity of each box C
max_boxes = 3  # Upper bound on the number of boxes N

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Decision variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)  # Assignment of items to boxes
model.y = pyo.Var(model.J, domain=pyo.Binary)  # Usage of boxes

# Objective function
model.obj = pyo.Objective(expr=pyo.summation(model.y), sense=pyo.minimize)

# Constraints
# Each item must be assigned to exactly one box
model.constraint1 = pyo.ConstraintList()
for i in model.I:
    model.constraint1.add(expr=sum(model.x[i, j] for j in model.J) == 1)

# Capacity constraint of each box
model.constraint2 = pyo.ConstraintList()
for j in model.J:
    model.constraint2.add(
        expr=sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C * model.y[j]
    )

# Limit on the number of boxes
model.constraint3 = pyo.Constraint(expr=pyo.summation(model.y) <= model.N)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the solution
print("Optimal solution:")
for j in model.J:
    if pyo.value(model.y[j]) > 0:
        print(f"Box {j}:")
        for i in model.I:
            if pyo.value(model.x[i, j]) > 0:
                print(f"  Item {i} (weight={model.w[i]})")
```

This code defines the model, sets, parameters, decision variables, objective function, and constraints as described in the problem statement. 

**Explanation of sample data:**
* **items:**  A list of item identifiers (e.g., 'A', 'B', etc.)
* **boxes:** A list of box identifiers (e.g., 'Box1', 'Box2', etc.)
* **weights:** A dictionary mapping each item to its weight.
* **capacity:**  The capacity of each box.
* **max_boxes:** The maximum number of boxes allowed. 

You can replace the sample data with your actual data to solve your specific instance of the problem. 


## 4. Input problem data and try running the generated code

In [604]:
import pyomo.environ as pyo

# Sample data (You can replace this with your actual data)
items = ['A', 'B', 'C', 'D']  # Set of items I
boxes = ['Box1', 'Box2', 'Box3']  # Set of boxes J
weights = {'A': 5, 'B': 3, 'C': 7, 'D': 2}  # Weights of items w_i
capacity = 10  # Capacity of each box C
max_boxes = 3  # Upper bound on the number of boxes N

items =  range(1,25)
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
boxes = range(1,15)
capacity = 9
max_boxes = 15


# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Decision variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)  # Assignment of items to boxes
model.y = pyo.Var(model.J, domain=pyo.Binary)  # Usage of boxes

# Objective function
model.obj = pyo.Objective(expr=pyo.summation(model.y), sense=pyo.minimize)

# Constraints
# Each item must be assigned to exactly one box
model.constraint1 = pyo.ConstraintList()
for i in model.I:
    model.constraint1.add(expr=sum(model.x[i, j] for j in model.J) == 1)

# Capacity constraint of each box
model.constraint2 = pyo.ConstraintList()
for j in model.J:
    model.constraint2.add(
        expr=sum(model.w[i] * model.x[i, j] for i in model.I) <= model.C * model.y[j]
    )

# Limit on the number of boxes
model.constraint3 = pyo.Constraint(expr=pyo.summation(model.y) <= model.N)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the solution
print("Optimal solution:")
for j in model.J:
    if pyo.value(model.y[j]) > 0:
        print(f"Box {j}:")
        for i in model.I:
            if pyo.value(model.x[i, j]) > 0:
                print(f"  Item {i} (weight={model.w[i]})")

Optimal solution:
Box 1:
  Item 23 (weight=8)
Box 2:
  Item 3 (weight=2)
  Item 21 (weight=7)
Box 3:
  Item 22 (weight=7)
Box 4:
  Item 4 (weight=2)
  Item 16 (weight=5)
Box 5:
  Item 2 (weight=2)
  Item 20 (weight=6)
Box 6:
  Item 1 (weight=2)
  Item 6 (weight=3)
  Item 12 (weight=4)
Box 7:
  Item 10 (weight=4)
  Item 14 (weight=5)
Box 8:
  Item 7 (weight=4)
  Item 18 (weight=5)
Box 9:
  Item 9 (weight=4)
  Item 17 (weight=5)
Box 10:
  Item 11 (weight=4)
  Item 15 (weight=5)
Box 11:
  Item 5 (weight=3)
  Item 19 (weight=6)
Box 12:
  Item 24 (weight=8)
Box 13:
  Item 8 (weight=4)
  Item 13 (weight=5)


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [605]:
print(response.text)

### Sets:
*  $I$: Set of items
*  $J$: Set of boxes

### Parameters:
*  $w_i$: Weight of item $i \in I$
*  $C$: Capacity of each box 
*  $N$: Upper bound on the number of boxes 

### Decision Variables:
*  $x_{ij}$: Binary variable equal to 1 if item  $i \in I$ is assigned to box $j \in J$, and 0 otherwise 
*  $y_j$: Binary variable equal to 1 if box $j \in J$ is used, and 0 otherwise 



In [606]:
print(response2.text)

Minimize $\sum_{j \in J} y_j$ 



In [607]:
print(response3.text)

## Constraints:

1. **Each item must be assigned to exactly one box:**
   $$ \sum_{j \in J} x_{ij} = 1, \quad \forall i \in I$$

2. **Capacity constraint of each box:**
   $$ \sum_{i \in I} w_i x_{ij} \le C y_j, \quad \forall j \in J$$

3. **Limit on the number of boxes:**
   $$ \sum_{j \in J} y_j \le N$$

4. **Decision variables are binary:**
   $$ x_{ij} \in \{0,1\}, \quad \forall i \in I, \forall j \in J$$
   $$ y_j \in \{0,1\}, \quad \forall j \in J $$ 



In [608]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data (You can replace this with your actual data)
items = ['A', 'B', 'C', 'D']  # Set of items I
boxes = ['Box1', 'Box2', 'Box3']  # Set of boxes J
weights = {'A': 5, 'B': 3, 'C': 7, 'D': 2}  # Weights of items w_i
capacity = 10  # Capacity of each box C
max_boxes = 3  # Upper bound on the number of boxes N

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.J = pyo.Set(initialize=boxes)

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=capacity)
model.N = pyo.Param(initialize=max_boxes)

# Decision variables
model.x = pyo.Var(model.I, model.J, domain=pyo.Binary)  # Assignment of items to boxes
model.y = pyo.Var(model.J, domain=pyo.Binary)  # Usage of boxes

# Objective function
model.obj = pyo.Objective(expr=pyo.summation(model.y), sense=pyo.minimize)

# Constraints
# Each item must be assigned to exactly one box
model.constraint1 = pyo.ConstraintList()
for i 